In [1]:
'''En este archivo tendremos implementado el modelo de clasificacion seleccionado para el caso ,el cual 
viene siendo el de mayor precisión, en este caso el CATBOOST with optuna '''
#iniciamos colocando las librerias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns



from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder,StandardScaler,PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

import optuna
from catboost import CatBoostClassifier

from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer

from sklearn.model_selection import KFold, cross_val_predict, train_test_split,GridSearchCV,cross_val_score
from sklearn.metrics import accuracy_score,classification_report

#importing plotly and cufflinks in offline mode
import cufflinks as cf
import plotly.offline
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)


import plotly 
import plotly.express as px
import plotly.graph_objs as go
import plotly.offline as py
from plotly.offline import iplot
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

import missingno as msno

import warnings
warnings.filterwarnings("ignore")

In [2]:
#Seguimos con la carga de datos del heart.cvs

pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',900)

pd.set_option('display.max_colwidth',200)

df = pd.read_csv('heart.csv')
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [3]:
#Implementacion del modelo de predicción que usaremos para predecir  a los nuevos pacientes
accuracy =[]
model_names =[]


X= df.drop('HeartDisease', axis=1)
y= df['HeartDisease']
categorical_features_indices = np.where(X.dtypes != float)[0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = CatBoostClassifier(verbose=False,random_state=0,
                          objective= 'CrossEntropy',
    colsample_bylevel= 0.04292240490294766,
    depth= 10,
    boosting_type= 'Plain',
    bootstrap_type= 'MVS')

model.fit(X_train, y_train,cat_features=categorical_features_indices,eval_set=(X_test, y_test))
y_pred = model.predict(X_test)
accuracy.append(round(accuracy_score(y_test, y_pred),4))
print(classification_report(y_test, y_pred))

model_names = ['Catboost_tuned']
result_df6 = pd.DataFrame({'Accuracy':accuracy}, index=model_names)
result_df6

              precision    recall  f1-score   support

           0       0.88      0.90      0.89       112
           1       0.93      0.91      0.92       164

    accuracy                           0.91       276
   macro avg       0.90      0.91      0.91       276
weighted avg       0.91      0.91      0.91       276



,Accuracy
Catboost_tuned,0.9094


In [4]:
def predict_heart_disease(model, patient_data):
    # Convierte los datos del paciente a un DataFrame de pandas
    patient_df = pd.DataFrame([patient_data])

    # Realiza la predicción utilizando el modelo entrenado
    prediction = model.predict(patient_df)

    # Traduce el resultado a una etiqueta más legible
    if prediction[0] == 1:
        result_label = 'Tiene riesgo de ataque cardíaco'
    else:
        result_label = 'No tiene riesgo de ataque cardíaco'

    # Devuelve el resultado como un diccionario
    result = {
        'patient_data': patient_data,
        'prediction_label': result_label
    }

    return result

# Datos de un nuevo paciente para la predicción
new_patient_data = {
    'Age': 45,
    'Sex': 'F',
    'ChestPainType': 'NAP',
    'RestingBP': 130,
    'Cholesterol': 220,
    'FastingBS': 1,
    'RestingECG': 'Normal',
    'MaxHR': 150,
    'ExerciseAngina': 'N',
    'Oldpeak': 1.0,
    'ST_Slope': 'Flat'
}

# Utiliza la función para predecir el riesgo de ataque cardíaco para el nuevo paciente
result = predict_heart_disease(model, new_patient_data)

# Muestra los resultados
print(result)
print(result_df6)

{'patient_data': {'Age': 45, 'Sex': 'F', 'ChestPainType': 'NAP', 'RestingBP': 130, 'Cholesterol': 220, 'FastingBS': 1, 'RestingECG': 'Normal', 'MaxHR': 150, 'ExerciseAngina': 'N', 'Oldpeak': 1.0, 'ST_Slope': 'Flat'}, 'prediction_label': 'No tiene riesgo de ataque cardíaco'}
                Accuracy
Catboost_tuned    0.9094


In [5]:
new_patient_data_risk = {
    'Age': 60,
    'Sex': 'M',
    'ChestPainType': 'ATA',
    'RestingBP': 140,
    'Cholesterol': 250,
    'FastingBS': 1,
    'RestingECG': 'ST',
    'MaxHR': 130,
    'ExerciseAngina': 'Y',
    'Oldpeak': 2.0,
    'ST_Slope': 'Up'
}

result_risk = predict_heart_disease(model, new_patient_data_risk)
print(result_risk)


{'patient_data': {'Age': 60, 'Sex': 'M', 'ChestPainType': 'ATA', 'RestingBP': 140, 'Cholesterol': 250, 'FastingBS': 1, 'RestingECG': 'ST', 'MaxHR': 130, 'ExerciseAngina': 'Y', 'Oldpeak': 2.0, 'ST_Slope': 'Up'}, 'prediction_label': 'Tiene riesgo de ataque cardíaco'}


In [6]:
import joblib
# Guardar el modelo en un archivo
joblib.dump(model, 'modelo_riesgo_cardiaco.joblib')

['modelo_riesgo_cardiaco.joblib']